## 필요한 모듈 import

In [1]:
# numpy 로드하여 np로 사용
# pandas 로드하여 pd로 사용
# matplotlib.pyplot 로드하여 plt로 사용
# seaborn 로드하여 sns로 사용
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

In [ ]:
# font name을 알고 있다면 생략가능
#한글폰트 path 설정
font_path = 'C:\\Windows\\Fonts\\malgun.ttf'
#font의 파일정보로 font name 을 알아내기
font_prop = fm.FontProperties(fname=font_path).get_name()
print(font_prop) # 폰트이름 확인하기

In [3]:
# matplotlib의 rc(run command) 함수를 사용해서 font name(Malgun Gothic) 설정
matplotlib.rc('font', family=font_prop)

### Mcdonalds_data.csv 의 데이터 확인

In [ ]:
import pandas as pd

# 상대경로 기준: check.ipynb → code/visualization/ → ../ → ../ → data/
df = pd.read_csv("../../data/Mcdonalds_data.csv")

# 데이터 확인
print(df.head())
print(df.columns)

### 번역을 위한 모듈 import

In [ ]:
# 번역을 위한 모듈 설치치
# %pip install deepl
# %pip install python-dotenv deepl pandas

In [ ]:
%pip show deepl
%pip show python-dotenv deepl pandas

In [ ]:
# Mcdonalds_data.csv에서 item 항목의 영어 메뉴명을 한글로 번역
# deepl api를 활용함

import deepl
import pandas as pd
import time
from dotenv import load_dotenv
import os
import sys # sys 모듈 추가

# ✅ .env 파일 로드
load_dotenv()

# ✅ API 키 불러오기
auth_key = os.getenv("DEEPL_API_KEY")

# --- 디버깅 코드 추가 ---
# print(f"Loaded API Key: {auth_key}") # 로드된 키 값 확인
# if not auth_key:
#     print("오류: .env 파일에서 DEEPL_API_KEY를 찾을 수 없거나 환경 변수가 설정되지 않았습니다.")
#     sys.exit() # 키가 없으면 프로그램 종료
# --- 디버깅 코드 끝 ---

try:
    translator = deepl.Translator(auth_key)
except Exception as e:
    print(f"DeepL Translator 초기화 중 오류 발생: {e}")
    sys.exit() # 초기화 실패 시 종료

# ✅ 메뉴 데이터 로드
try:
    df = pd.read_csv("../../data/Mcdonalds_data.csv")
except FileNotFoundError:
    print("오류: ../../data/Mcdonalds_data.csv 파일을 찾을 수 없습니다. 파일 경로를 확인하세요.")
    sys.exit()

menu_items = df["Item"].unique().tolist()

# ✅ 번역 결과 저장용 딕셔너리
menu_eng_to_kor = {}

# ✅ 번역 실행
print("번역 작업을 시작합니다...")
for i, item in enumerate(menu_items):
    # 빈 문자열이나 공백만 있는 항목은 건너뛰기
    if not item or item.strip() == "":
        print(f"[{i+1}/{len(menu_items)}] 건너뜀 (빈 항목): '{item}'")
        menu_eng_to_kor[item] = "" # 빈 값으로 저장하거나 다른 방식으로 처리
        continue

    try:
        result = translator.translate_text(item, source_lang="EN", target_lang="KO")
        menu_eng_to_kor[item] = result.text
        print(f"[{i+1}/{len(menu_items)}] {item} → {result.text}")
        time.sleep(0.5)  # API 호출 속도 제한 준수
    except deepl.exceptions.AuthorizationException as e:
        print(f"[인증 오류] {item} → {e}. API 키가 유효한지, 계정 상태를 확인하세요.")
        menu_eng_to_kor[item] = item # 에러 시 원본 저장
        # 인증 오류 발생 시 반복 중단 (선택 사항)
        # print("치명적인 인증 오류로 번역을 중단합니다.")
        # break
    except deepl.exceptions.QuotaExceededException as e:
         print(f"[쿼터 초과] {item} → {e}. DeepL API 사용량 한도를 초과했습니다.")
         menu_eng_to_kor[item] = item
         # 쿼터 초과 시 반복 중단
         print("API 사용량 한도 초과로 번역을 중단합니다.")
         break
    except Exception as e:
        print(f"[일반 오류] {item} → {e}")
        menu_eng_to_kor[item] = item  # 에러 시 원본 저장

# ✅ 결과 저장
if menu_eng_to_kor: # 번역 결과가 있을 경우에만 저장
    translated_df = pd.DataFrame(list(menu_eng_to_kor.items()), columns=["English Name", "Korean Translation"])
    try:
        translated_df.to_csv("translated_menu_deepl.csv", index=False, encoding='utf-8-sig') # UTF-8 BOM 인코딩 명시
        print("✅ DeepL 번역 완료! translated_menu_deepl.csv 저장됨.")
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")
else:
    print("번역된 항목이 없어 CSV 파일을 저장하지 않습니다.")

Loaded API Key: c7b3d072-b43f-4012-85e7-9201aa19b049:fx
번역 작업을 시작합니다...
[1/260] Egg McMuffin → 에그 맥머핀
[2/260] Egg White Delight → 달걀 흰자 딜라이트
[3/260] Sausage McMuffin → 소시지 맥머핀
[4/260] Sausage McMuffin with Egg → 소시지 맥머핀과 계란
[5/260] Sausage McMuffin with Egg Whites → 달걀 흰자를 곁들인 소시지 맥머핀
[6/260] Steak & Egg McMuffin → 스테이크 & 에그 맥머핀
[7/260] Bacon, Egg & Cheese Biscuit (Regular Biscuit) → 베이컨, 에그 & 치즈 비스킷 (일반 비스킷)
[8/260] Bacon, Egg & Cheese Biscuit (Large Biscuit) → 베이컨, 에그 & 치즈 비스킷 (라지 비스킷)
[9/260] Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit) → 베이컨, 달걀, 치즈와 달걀 흰자 비스킷 (일반 비스킷)
[10/260] Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit) → 베이컨, 달걀, 치즈와 달걀 흰자 비스킷 (라지 비스킷)
[11/260] Sausage Biscuit (Regular Biscuit) → 소시지 비스킷 (일반 비스킷)
[12/260] Sausage Biscuit (Large Biscuit) → 소시지 비스킷 (라지 비스킷)
[13/260] Sausage Biscuit with Egg (Regular Biscuit) → 계란 소시지 비스킷 (일반 비스킷)
[14/260] Sausage Biscuit with Egg (Large Biscuit) → 계란 소시지 비스킷 (라지 비스킷)
[15/260] Sausage Biscuit with Eg